# VLM을 사용하여 이미지와 문서로부터 구조화된 데이터 생성하기

HuggingFaceTB의 SmolVLM-Instruct 모델을 사용하여 문서로부터 구조화된 정보를 추출합니다. Hugging Face Transformers 라이브러리를 통해 실행되며, [Outlines](https://github.com/dottxt-ai/outlines) 라이브러리를 사용하여 토큰 샘플링 확률을 제한함으로써 **정형화된 출력 형식을 생성**하도록 돕습니다.

> 이 방식은 [Outlines tutorial](https://dottxt-ai.github.io/outlines/latest/cookbook/atomic_caption/) 기반으로 진행합니다.

## Dependencies and imports

첫번째로, 필수 라이브러리를 설치합니다.

In [16]:
%pip install accelerate outlines transformers torch flash-attn datasets sentencepiece

필수 라이브러리

In [17]:
import outlines # 구조화된 결과 생성을 돕는 라이브러리
import torch

from datasets import load_dataset
from outlines.models.transformers_vision import transformers_vision
from transformers import AutoModelForImageTextToText, AutoProcessor
from pydantic import BaseModel

## 모델 초기화

먼저 [HuggingFaceTB/SmolVLM-Instruct](https://huggingface.co/HuggingFaceTB/SmolVLM-Instruct) 모델을 초기화합니다. Outline은 모델 클래스와 프로세스 클래스를 정보를 명시적으로 전달해야 하므로, 해당 클래스를 반환하는 함수를 작성하여 사용하는 방식으로 예제를 구성합니다. 다른 방법으로는 [Hub repo files](https://huggingface.co/HuggingFaceTB/SmolVLM-Instruct/tree/main)에서 직접 모델과 토크나이저를 보고 직접 클래스를 지정하여 가져오는 방식도 사용할 수 있습니다.

In [18]:
model_name = "HuggingFaceTB/SmolVLM-Instruct"


def get_model_and_processor_class(model_name: str):
    model = AutoModelForImageTextToText.from_pretrained(model_name)
    processor = AutoProcessor.from_pretrained(model_name)
    classes = model.__class__, processor.__class__
    del model, processor
    return classes


model_class, processor_class = get_model_and_processor_class(model_name)

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

model = transformers_vision(
    model_name,
    model_class=model_class,
    # device=device,
    model_kwargs={"torch_dtype": torch.bfloat16, "device_map": "auto"},
    # processor_kwargs={"device": device},
    processor_class=processor_class,
)

In [19]:
print(processor_class)
print(type(processor_class))

<class 'transformers.models.idefics3.processing_idefics3.Idefics3Processor'>
<class 'type'>


## 구조화된 출력 생성

이제 모델이 구조화된 출력을 생성하는 함수를 정의합니다. 이미지와 해당 이미지에 대한 질문-선택 응답, 거부 응답 쌍으로 구성되어 있는 [openbmb/RLAIF-V-Dataset](https://huggingface.co/datasets/openbmb/RLAIF-V-Dataset)을 사용합니다. 이 데이터세트를 활용할 수 있으나, 여기에 추가적으로 text=image-to-text 데이터를 생성하여 우리만의 **구조화된 데이터세트**를 만들고, 이후 모델을 fine-tuning 하는데 활용할 계획입니다. 구체적으로 이미지에 대해 캡션, 질문, 간단한 품질 판단 태그를 생성하는 작업을 수행할 것입니다.

In [20]:
class ImageData(BaseModel):
    quality: str
    description: str
    question: str

structured_generator = outlines.generate.json(model, ImageData)

프롬프트를 작성합니다.

In [21]:
prompt = """
You are an image analysis assisant.

Provide a quality tag, a description and a question.

The quality can either be "good", "okay" or "bad".
The question should be concise and objective.

Return your response as a valid JSON object.
""".strip()

이미지 데이터세트를 로드합니다.

In [22]:
dataset = load_dataset("openbmb/RLAIF-V-Dataset", split="train[:10]")
dataset

Dataset({
    features: ['ds_name', 'image', 'question', 'chosen', 'rejected', 'origin_dataset', 'origin_split', 'idx', 'image_path'],
    num_rows: 10
})

이미지로부터 구조화된 정보를 추출하는 함수를 정의합니다. `apply_chat_template` 메소드를 사용하여 프롬프트 형식을 지정하고, 이미지와 함께 형식을 모델에 전달합니다.

In [23]:
def extract(row):
    messages = [
        {
            "role": "user",
            "content": [{"type": "image"}, {"type": "text", "text": prompt}],
        },
    ]

    formatted_prompt = model.processor.apply_chat_template( # apply_chat_template: Hugging Face에서 제공하는 chat 템플릿 적용 함수. 메시지를 모델이 이해할 수 있는 포맷으로 변환.
        messages, add_generation_prompt=True
    )

    result = structured_generator(formatted_prompt, [row["image"]])
    row['synthetic_question'] = result.question
    row['synthetic_description'] = result.description
    row['synthetic_quality'] = result.quality
    return row


dataset = dataset.map(lambda x: extract(x))
dataset

/usr/local/lib/python3.11/dist-packages/dill/_dill.py:414: PicklingWarning: Cannot locate reference to <class '__main__.ImageData'>.
  StockPickler.save(self, obj, save_persistent_id)
/usr/local/lib/python3.11/dist-packages/dill/_dill.py:414: PicklingWarning: Cannot pickle <class '__main__.ImageData'>: __main__.ImageData has recursive self-references that trigger a RecursionError.
  StockPickler.save(self, obj, save_persistent_id)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['ds_name', 'image', 'question', 'chosen', 'rejected', 'origin_dataset', 'origin_split', 'idx', 'image_path', 'synthetic_question', 'synthetic_description', 'synthetic_quality'],
    num_rows: 10
})

In [24]:
dataset

Dataset({
    features: ['ds_name', 'image', 'question', 'chosen', 'rejected', 'origin_dataset', 'origin_split', 'idx', 'image_path', 'synthetic_question', 'synthetic_description', 'synthetic_quality'],
    num_rows: 10
})

In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

새로 만든 데이터세트를 Hugging Face Hub에 업로드 해봅시다.

In [ ]:
from huggingface_hub import create_repo
create_repo("structured-generation-mydata", repo_type="dataset")


In [ ]:
# dataset.push_to_hub("davidberenstein1957/structured-generation-information-extraction-vlms-openbmb-RLAIF-V-Dataset", split="train")

dataset.push_to_hub("yeomjihyun/structured-generation-mydata", split="train")

<iframe
  src="https://huggingface.co/datasets/davidberenstein1957/structured-generation-information-extraction-vlms-openbmb-RLAIF-V-Dataset/embed/viewer/default/train?row=3"
  frameborder="0"
  width="100%"
  height="560px"
></iframe>

결과가 완벽하지 않다면 다른 모델과 프롬프트를 실험해보기에 좋은 출발점입니다!

## 결론

이번 튜토리얼에서는 비전-언어 모델을 사용해 문서에서 구조화된 정보를 추출하는 방법을 알아보았습니다.
비슷한 추출 방법을 활용하면 PDF 문서를 pdf2image와 같은 도구를 사용해 이미지로 변환한 후, 각 페이지 이미지에 대해 정보를 추출할 수 있습니다.

```python
pdf_path = "path/to/your/pdf/file.pdf"
pages = convert_from_path(pdf_path)
for page in pages:
    extract_objects = extract_objects(page, prompt)
```

## 다음 단계
[Outlines](https://github.com/outlines-ai/outlines) 라이브러리를 참고하여 다양한 사용법과 파라미터를 더 알아보세요.

여러분의 모델을 사용해, 직접 사용 사례에 맞는 정보를 추출해보세요.

문서에서 구조화된 정보를 추출하는 다른 방법도 탐색해보세요.